In [1]:
import numpy as np
import pandas as pd

import requests
from bs4 import BeautifulSoup

In [2]:
df = pd.read_csv('taxappeal.csv')

In [3]:
df.head()

,21303140150000,2718,E 77TH ST,CHICAGO,100,2-02,$10385,Unnamed: 7
0,21303140160000,2722,E 77TH ST,CHICAGO,100,2-02,$10132,NaN
1,21303140170000,2726,E 77TH ST,CHICAGO,100,2-02,$9766,NaN
2,21303140180000,2728,E 77TH ST,CHICAGO,100,2-02,$10974,NaN
3,21303230060000,7727,S SAGINAW AVE,CHICAGO,100,2-02,$12301,NaN
4,21303310080000,7823,S MARQUETTE AVE,CHICAGO,100,2-02,$11692,NaN


In [4]:
df.columns = ['PIN', 'Address', 'Street', 'City', 'Neighborhood', 'Class', 'AV', 'Unit']

In [5]:
df.drop(columns= ['City', 'Neighborhood', 'Class'], inplace=True)

In [6]:
def remove_sign(value):
        return value[1:]

In [7]:
df['AV'] = df['AV'].apply(remove_sign)

In [8]:
df['AV'] = df['AV'].astype(int)

In [9]:
df.head()

,PIN,Address,Street,AV,Unit
0,21303140160000,2722,E 77TH ST,10132,NaN
1,21303140170000,2726,E 77TH ST,9766,NaN
2,21303140180000,2728,E 77TH ST,10974,NaN
3,21303230060000,7727,S SAGINAW AVE,12301,NaN
4,21303310080000,7823,S MARQUETTE AVE,11692,NaN


In [10]:
df[(df['AV'] > 9000) & (df['AV'] < 9050)].sort_values('AV')

,PIN,Address,Street,AV,Unit
52,21311130420000,8054,S MARQUETTE AVE,9006,NaN
389,21313270340000,8634,S COLFAX AVE,9006,NaN
392,21313280050000,8613,S COLFAX AVE,9007,NaN
337,21313190340000,8538,S COLFAX AVE,9013,NaN
229,21312320300000,8218,S SOUTH SHORE DR,9014,NaN
567,26061110220000,8810,S COLFAX AVE,9016,NaN
65,21311150230000,8020,S BURNHAM AVE,9021,NaN
166,21312170070000,8115,S MUSKEGON AVE,9021,NaN
309,21313140300000,8432,S MANISTEE AVE,9026,NaN
125,21312060110000,7937,S COLES AVE,9044,NaN


In [11]:
pin_numbers = list(df['PIN'])

In [12]:
# Scraping the Data Set
pin_info = []
sqft_info = []
age_info = []
lav_info = []
bav_info = []
bldg_info =[]
extr_info = []

url="http://cookcountyassessor.com/Property.aspx?mode=details&pin="
for i in pin_numbers:
    page = requests.get(url+str(i))
    soup = BeautifulSoup(page.content, 'html.parser')
    
    pin = soup.find("span", id="ctl00_phArticle_ctlPropertyDetails_lblPropInfoPIN").get_text()
    pin_info.append(pin)
    
    sqft = soup.find("span", id="ctl00_phArticle_ctlPropertyDetails_lblPropInfoSqFt").get_text()
    sqft_info.append(sqft)
    
    age = soup.find("span", id="ctl00_phArticle_ctlPropertyDetails_lblPropCharAge").get_text()
    age_info.append(age)
    
    lav = soup.find("span", id="ctl00_phArticle_ctlPropertyDetails_lblAsdValLandFirstPass").get_text()
    lav_info.append(lav)
    
    bav = soup.find("span", id="ctl00_phArticle_ctlPropertyDetails_lblAsdValBldgFirstPass").get_text()
    bav_info.append(bav)
    
    bldg = soup.find("span", id="ctl00_phArticle_ctlPropertyDetails_lblPropCharBldgSqFt").get_text()
    bldg_info.append(bldg)
    
    extr = soup.find("span", id="ctl00_phArticle_ctlPropertyDetails_lblPropCharExtConst").get_text()
    extr_info.append(extr)
    
gs = pd.DataFrame({'PIN': pin_info,
                   'Land Sqft': sqft_info,
                   'Age': age_info,
                   'Land AV': lav_info,
                   'Bldg AV': bav_info,
                   'Bldg Sqft': bldg_info,
                   'Exterior': extr_info
                  })

gs.to_csv('out.csv')

In [13]:
gs = pd.read_csv('out.csv')

In [14]:
gs.head()

,Unnamed: 0,PIN,Land Sqft,Age,Land AV,Bldg AV,Bldg Sqft,Exterior
0,0,21-30-314-016-0000,"2,910",102,"2,910","7,222",736,Masonry
1,1,21-30-314-017-0000,"2,910",106,"2,910","6,856",736,Frame
2,2,21-30-314-018-0000,"2,910",106,"2,910","8,064",883,Masonry
3,3,21-30-323-006-0000,"8,350",125,"8,350","3,951",882,Frame
4,4,21-30-331-008-0000,"4,770",110,"4,770","6,922",972,Frame


In [15]:
gs.columns

Index(['Unnamed: 0', 'PIN', 'Land Sqft', 'Age', 'Land AV', 'Bldg AV',
       'Bldg Sqft', 'Exterior'],
      dtype='object')

In [16]:
gs.drop(columns=['Unnamed: 0'], inplace=True)

In [17]:
def remove_char(value):
    return value.replace("-", "")

def remove_char2(value):
    return value.replace(",", "")

In [18]:
gs['PIN'] = gs['PIN'].apply(remove_char)
gs['Bldg AV'] = gs['Bldg AV'].apply(remove_char2)
gs['Land AV'] = gs['Land AV'].apply(remove_char2)
gs['Land Sqft'] = gs['Land Sqft'].apply(remove_char2)

In [19]:
gs['Age'] = gs['Age'].astype(int)
gs['Bldg AV'] = gs['Bldg AV'].astype(int)
gs['Bldg Sqft'] = gs['Bldg Sqft'].astype(int)
gs['Land AV'] = gs['Land AV'].astype(int)
gs['PIN'] = gs['PIN'].astype(int)
gs['Land Sqft'] = gs['Land Sqft'].astype(int)

In [20]:
gs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 604 entries, 0 to 603
Data columns (total 7 columns):
PIN          604 non-null int64
Land Sqft    604 non-null int64
Age          604 non-null int64
Land AV      604 non-null int64
Bldg AV      604 non-null int64
Bldg Sqft    604 non-null int64
Exterior     604 non-null object
dtypes: int64(6), object(1)
memory usage: 33.1+ KB


In [21]:
def tav(row):
    return row['Bldg AV'] + row['Land AV']

def bldg_mv_per_sqft(row):
    return round(row['Bldg AV'] * 10 / row['Bldg Sqft'], 2)

In [22]:
gs['Total AV'] = gs.apply(tav, axis=1)

In [23]:
gs.head()

,PIN,Land Sqft,Age,Land AV,Bldg AV,Bldg Sqft,Exterior,Total AV
0,21303140160000,2910,102,2910,7222,736,Masonry,10132
1,21303140170000,2910,106,2910,6856,736,Frame,9766
2,21303140180000,2910,106,2910,8064,883,Masonry,10974
3,21303230060000,8350,125,8350,3951,882,Frame,12301
4,21303310080000,4770,110,4770,6922,972,Frame,11692


In [24]:
gs['persqft'] = gs.apply(bldg_mv_per_sqft, axis=1)
gs.head()

,PIN,Land Sqft,Age,Land AV,Bldg AV,Bldg Sqft,Exterior,Total AV,persqft
0,21303140160000,2910,102,2910,7222,736,Masonry,10132,98.12
1,21303140170000,2910,106,2910,6856,736,Frame,9766,93.15
2,21303140180000,2910,106,2910,8064,883,Masonry,10974,91.33
3,21303230060000,8350,125,8350,3951,882,Frame,12301,44.80
4,21303310080000,4770,110,4770,6922,972,Frame,11692,71.21


In [25]:
mt = pd.merge(df, gs, on='PIN')

In [26]:
mt.head()

,PIN,Address,Street,AV,Unit,Land Sqft,Age,Land AV,Bldg AV,Bldg Sqft,Exterior,Total AV,persqft
0,21303140160000,2722,E 77TH ST,10132,NaN,2910,102,2910,7222,736,Masonry,10132,98.12
1,21303140170000,2726,E 77TH ST,9766,NaN,2910,106,2910,6856,736,Frame,9766,93.15
2,21303140180000,2728,E 77TH ST,10974,NaN,2910,106,2910,8064,883,Masonry,10974,91.33
3,21303230060000,7727,S SAGINAW AVE,12301,NaN,8350,125,8350,3951,882,Frame,12301,44.80
4,21303310080000,7823,S MARQUETTE AVE,11692,NaN,4770,110,4770,6922,972,Frame,11692,71.21


In [29]:
#Same Block
mt[(mt['Address'] >= 7700) & (mt['Street'] == 'S MUSKEGON AVE')]

,PIN,Address,Street,AV,Unit,Land Sqft,Age,Land AV,Bldg AV,Bldg Sqft,Exterior,Total AV,persqft
7,21304000280000,7732,S MUSKEGON AVE,10724,NaN,5566,122,5566,5158,880,Frame,10724,58.61
8,21304070230000,7826,S MUSKEGON AVE,10765,NaN,5962,130,5962,4803,884,Frame,10765,54.33
9,21304070370000,7846,S MUSKEGON AVE,9846,NaN,4134,138,4134,5712,902,Frame,9846,63.33
10,21304080040000,7815,S MUSKEGON AVE,10228,NaN,3975,130,3975,6253,900,Frame,10228,69.48
108,21312000320000,7932,S MUSKEGON AVE,9945,NaN,3100,110,3100,6845,893,Frame,9945,76.65
109,21312000370000,7948,S MUSKEGON AVE,9222,NaN,3100,125,3100,6122,910,Frame,9222,67.27
110,21312010090000,7931,S MUSKEGON AVE,9750,NaN,4650,128,4650,5100,903,Frame,9750,56.48
128,21312080260000,8008,S MUSKEGON AVE,8556,NaN,3100,115,3100,5456,672,Frame,8556,81.19
129,21312090030000,8005,S MUSKEGON AVE,8641,NaN,3100,118,3100,5541,698,Frame,8641,79.38
130,21312090060000,8015,S MUSKEGON AVE,8558,NaN,3100,125,3100,5458,784,Frame,8558,69.62


In [30]:
#Similar Homes
mt[(mt['Age'] > 120) & (mt['Bldg Sqft'] >= 880) & (mt['Bldg Sqft'] <= 890) & (mt['persqft'] <= 70)]

,PIN,Address,Street,AV,Unit,Land Sqft,Age,Land AV,Bldg AV,Bldg Sqft,Exterior,Total AV,persqft
3,21303230060000,7727,S SAGINAW AVE,12301,NaN,8350,125,8350,3951,882,Frame,12301,44.80
7,21304000280000,7732,S MUSKEGON AVE,10724,NaN,5566,122,5566,5158,880,Frame,10724,58.61
8,21304070230000,7826,S MUSKEGON AVE,10765,NaN,5962,130,5962,4803,884,Frame,10765,54.33
126,21312080120000,8029,S BURNHAM AVE,9200,NaN,3100,125,3100,6100,882,Frame,9200,69.16
160,21312160180000,8143,S BURNHAM AVE,8520,NaN,3125,122,3125,5395,880,Frame,8520,61.31
392,21313280050000,8613,S COLFAX AVE,9007,NaN,3125,122,3125,5882,880,Frame,9007,66.84
419,21314010050000,8342,S BALTIMORE AVE,8222,NaN,2875,125,2875,5347,888,Frame,8222,60.21
537,21314270340000,8640,S BALTIMORE AVE,8662,1,3500,130,3500,5162,880,Frame,8662,58.66
573,26061120110000,8825,S COLFAX AVE,9630,NaN,3475,130,3475,6155,880,Frame,9630,69.94
584,26062080070000,8817,S ESCANABA AVE,9236,NaN,3475,137,3475,5761,890,Frame,9236,64.73
